Importing Libraries and ignoring warnings

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import filterwarnings
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D
from keras import models
from keras import layers
import tensorflow as tf
import os
import os.path
from pathlib import Path
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import regularizers
from keras.optimizers import RMSprop,Adam
import glob
from PIL import Image

# IGNORING UNNECESSARRY WARNINGS

filterwarnings("ignore",category=DeprecationWarning)
filterwarnings("ignore", category=FutureWarning) 
filterwarnings("ignore", category=UserWarning)

Organization of the Data

In [3]:
#Training Set

No_DataPath = Path("brain-tumor-detection/no/")
Yes_DataPath = Path("brain-tumor-detection/yes/")

No_jpgList = list(No_DataPath.glob(r"*.jpg"))
Yes_jpgList = list(Yes_DataPath.glob(r"*.jpg"))

In [ ]:
#Checking whether list works

print(No_jpgList[0:7])
print("-----------------------"*5)
print(Yes_jpgList[0:7])

In [6]:
#creating a list with both yes and no pictures

yn_List = []

for i in No_jpgList:
    yn_List.append(i)

for i in Yes_jpgList:
    yn_List.append(i)

print(yn_List[0:5])

[WindowsPath('brain-tumor-detection/no/no0.jpg'), WindowsPath('brain-tumor-detection/no/no1.jpg'), WindowsPath('brain-tumor-detection/no/no10.jpg'), WindowsPath('brain-tumor-detection/no/no100.jpg'), WindowsPath('brain-tumor-detection/no/no1000.jpg')]
